# Task 1 : Warm-up exercise
---

This notebook is intended to guide you through the (small number) of commands you need to load and run the model.

TO DO: This model describes something that can only move in a straight line moving through medium with some density of 'blocking' nodes.
It could represent light travelling through a medium containing opaque particles.
For example, the probability that a photon (particle of light) is detected after travelling through a certain thickness of gas. Note that we are not saying X out of every Y photons makes it through, just "at least one" makes it. Hence dependent on how many we start with...

At the end of the notebook, you will use the model to generate some data, which you will plot (by hand or with e.g. Excel) and compare to your theoretical prediction.

## Running the model
---

### Importing the code we need

If you see an error `name 'SquareLattice' is not defined`, or something similar, you may have forgotten to execute this cell.

In [ ]:
from p1b_pandemic.lattice import SquareLattice
from p1b_pandemic.model import PandemicModel

### Running an animation

In [ ]:
lattice = SquareLattice()
model = PandemicModel(lattice)

model.animate()

### Well that was rubbish...

Fortunately, this is not all we can do.
So far, we hare just using the rubbishy defaults.

Let's start by changing the size of the lattice so that it's longer in the horizontal direction.

In [ ]:
dimensions = (25, 100)

lattice = SquareLattice(dimensions)
model = PandemicModel(lattice)

model.animate()

### This still sucks...

Indeed it does. 
For one thing, the animation did not run for long enough.
We will increase the number of updates by providing a value when we run `model.animate()`.

In [ ]:
dimensions = (25, 100)

lattice = SquareLattice(dimensions)
model = PandemicModel(lattice)

model.animate(100)

### Better but still boring...

For now we will just add one more parameter - `vaccine_frac`.
This will 'freeze' some of the nodes on the lattice and prevent the virus from spreading past them.


In [ ]:
dimensions = (25, 100)
vaccine_frac = 0.1

lattice = SquareLattice(dimensions)
model = PandemicModel(lattice, vaccine_frac)

model.animate(100)

You'll (almost certainly) see that the virus gets stopped well before it reaches the opposite edge.
We will describe this situation as a 'failure to percolate'.

> We will say that a simulation **percolates** if any square on the right-most edge of the box turns red (i.e. becomes 'infected').

How low does `vaccine_frac` need to be before simulations start to percolate?
Experiment with different values by re-running the cell below.

In [ ]:
dimensions = (25, 100)
vaccine_frac = 0.03

lattice = SquareLattice(dimensions)
model = PandemicModel(lattice, vaccine_frac)

model.animate(100)

You should notice that, for values of `vaccine_frac` between about X and Y, the simulation sometimes percolates and sometimes doesn't.
This is because we are working with a *probabilistic model*.

So, the question "does the system percolate with `vaccine_frac = X`?" is not a valid one.
Instead, we should ask "what is the *probability* that the system percolates when `vaccine_frac = X`?"

## Calculating the theoretical percolation probability
---

As well as being a very unrealistic model of a pandemic, this model may strike you as being incredibly simple.
Indeed, it is not too tricky to calculate the probability that a simulation percolates by hand.

Actually, we can use this to our advantage, since we will be able to check that our model is working as expected by comparing simulation results to analytical results.
These sorts of 'sanity checks' are very important in modelling; before making your model overly complicated, it is very sensible to check that it reproduces a known result in a simple test-case.

Before doing the calculation let us define our variables.
Each node has a fixed probability of being immune, 

$$
\Pr(\text{node is immune}) \equiv q
$$

which is equal to `vaccine_frac`.
The lattice has $l_1$ rows and $l_2$ columns.

The first thing to notice is that, since the virus can only travel right (not up or down), each row is independent of every other row.

> Show that the probability $r$ that a *single row* percolates (i.e. the virus travels all the way from the left edge to the right edge) is
>
> $$\Pr(\text{single row percolates}) \equiv r = (1 - q)^{l_2}$$

> Next, show that the probability that *at least one* of the $l_1$ rows percolates is
>
> $$\Pr(\text{at least one row percolates}) \equiv p = 1 - (1 - r)^{l_1}$$

**Hint 1**: If $\Pr(A)$ describes the probability of event $A$ occuring and $\Pr(B)$ describes the probability of another, independent event occurring, then the probability that *both* $A$ *and* $B$ occur is 

$$
\Pr(\text{both } A \text{ and } B \text{ occur}) \equiv
\Pr(A \cap B) = \Pr(A) \Pr(B)
$$

**Hint 2**: you may find it useful to use the Binomial formula, which describes the probability of obtaining $k$ 'successes' out of $n$ trials, where each trial can result in either a success, with probability $\Pr(\text{success}) = p$, or a 'failure' with probability $\Pr(\text{failure}) = (1 - p)$.

$$
\Pr(k \text{ successes given } n \text{ trials}) \equiv
\Pr(k \mid n) = \frac{k!}{n!(n-k)!} p^k (1 - p)^{n-k}
$$

**Hint 3**: The probability of *at least one* success is one minus the probability of *zero* successes.

$$
\Pr(k > 1 \mid n) = 1 - \Pr(k = 0 \mid n)
$$


## Testing the model

The cell below will run a number, `repeats` of separate simulations and print out the fraction of these that percolated.

You should choose a number of repeats, and then run the cell for a range of different values of `vaccine_frac`, so that the lower values lead to approximately 100% of simulations percolating, and the higher values lead to simulations which almost never percolate.

Record your results in a table, making sure to keep a note of the theoretical prediction and the number of repeats.
Leave a column blank; you will need calculate errors on your data points shortly.
For example, the table could look like

| vaccine_frac ($q$) | Theoretical percolation prob ($p$) | Repeats ($n$) | Simulation results ($f$) | Error ($\sigma$) |
| --- | --- | --- | --- | --- |
| ... | ... | ... | ... | ... |

Once you've gathered enough data (20 points over the appropriate range should be sufficient), plot a graph (either by hand or using a plotting tool such as Excel), being careful to label your axes.
Do not join up the points!!!

As well as your data, you should plot the theoretical probabilities using the formula you proved earlier.
If you're plotting by hand, aim to plot 20 points and carefully sketch a continuous curve through them.


In [ ]:
dimensions = (100, 100)
vaccine_frac = 0.05  # change this value!
repeats = 100  # and this one!

lattice = SquareLattice(dimensions)
model = PandemicModel(lattice, vaccine_frac)

fraction_that_percolated = model.estimate_percolation_prob(repeats)
print(fraction_that_percolated)

## Calculating errors
---

We're now armed with a theoretical prediction, and a set of results .

However, it is important to realise that we can never *calculate* the percolation probability using simulations, but we can *estimate* it by running a large number of simulations and applying *statistics*.

More specifically, we will take the fraction of simulations that percolate to be an *estimator* for the true percolation probability.
This can be written

$$
f = \frac{1}{n} \sum_{i=1}^n b_i \tag{Fraction of simulations that percolate}
$$

where $b_i$ are equal to either 0 (for simulations which don't percolate) or 1 (for those that do).

Now the question becomes: how do we tell if our estimate $f$ is in agreement with the theoretical prediction $p$?
The answer is that we need to calculate the *standard error* on $f$, which we will denote $\sigma_f$.
This allows us to quantify the agreement by making statements such as
* _"the simulations agree with theory to within 1 standard error"_
* _"the model gives an answer that is $5\sigma$ away from the expected value"_

In this case, $(n \times f)$ follows a binomial distribution, which leads to the following formula for the standard error on $f$

$$
\sigma_f = \sqrt{\frac{p(1 - p)}{n}} \tag{Standard error}
$$

> **Bonus task**: derive this formula for the standard error on the mean of a Binomial distribution.

# TO DO give some hints. Maybe change symbol for SE to avoid confusion

## Python programming for the curious

...

However, this is purely to provide some explanation for those who want it.
It is **absolutely not part of of the laboratory to learn this terminology**, and to be totally clear, **no programming knowledge is required to complete this laboratory and gain full marks**.

> *Modules* are essentially just locations (files) where bits of code are grouped together and stored.
> We can load, or *import* some code from a module using the following syntax
> ```python
> from my_module import my_code
> ```

In the first cell we import two 'bits of code', `SquareLattice` and `PandemicModel`, from two different modules, `p1b_pandemic.lattice` and `p1b_pandemic.model`.

```python
from p1b_pandemic.lattice import SquareLattice
from p1b_pandemic.model import PandemicModel
```

So what are these 'bits of code' that we've imported?
In this case, `SquareLattice` and `PandemicModel` are both *classes*.

> *Classes* act as flexible templates, from which Python is able to create *objects*.
> The flexibility comes from the fact that we can specify certain *parameters* (like tuning the dials on a machine) when creating an object.
> The syntax for creating an object from a class is
> ```python
> my_object = MyClass(my_parameter, my_second_parameter, ...)
> ```

The next thing we do is create two objects: `lattice` and `model` from the two classes.

```python
lattice = SquareLattice()
model = PandemicModel(lattice)
```
`lattice` is actually given as a *parameter* to the `PandemicModel` class.
You might think it looks like there are no other parameters.
There are, but by not specifying them we just use the default values.

Once an object is created, we can interact with it.
In particular, we might want to ask it to
* Tell us the value of a certain *attribute* : `print(object.attribute)`
* Execute one of its *methods* : `object.method(argument_1, argument_2, ...)`
* Modify the value of an attribute : `object.attribute = new_value`

So when we run the animation,
```python
model.animate(n_days=100)
```
we're telling `model` to execute it's `animation` method, while passing the *argument* `n_days=100` that tells it to complete 100 iterations.


Now this all sounds incredibly abstract, but underneath the strange terminology are concepts that will seem very intuitive.
A 'real world' example will serve us well to explain how classes and objects work.

### A real world example
---

Imagine you are the chief baker in a bakery.
Every day you make a selection of delicious loaves, cakes, pastries from scratch.
However, your bakery has become so popular that you just can't keep up with demand, so you decide to invest in a machine that can make the dough and the batter for you.

In this story, the machine will be the analogue of the Python programming language.
It contains lots of complicated circuitry (*microcode*) but as the baker you rarely need to worry about all that.

The machine has two main settings: 'batter' (used to make cakes) and 'dough' (for bread).
These are two distinct *classes*.

Of course, there are different sorts of dough, depending on what kind of bread you're making.
So, after selecting the 'dough' setting, you are presented with a list of *parameters* - type of flour, amount of yeast, amount of salt... - which you can tune to get exactly the dough that you want.

You set the parameters to how you want them, and hit the 'start' button.
A couple of hours later, the dough pops out.
We now have an *object* that has been created based on the 'dough' class and the specific parameters you chose when starting the machine.

What's next?
You might want to leave the dough to prove for a while, perhaps shape it into a nice loaf-shape, and at some point it's going to need baking.
All of these things are analogous to *methods* belonging to an object.
You could also say that the dough has certain *attributes*, for example: 'time spent proving', 'shape', 'baked'.

Let's see how this story would play out in some Python code!
```python
from bread_maker import Dough, Batter  # import our classes

# Make the brown_dough object from the Dough class, using these specific parameters
brown_dough = Dough(water="225ml", flour="300g", flour_type="wholemeal", yeast="7g", salt="1tsp")

if brown_dough.shape is not "pretty":
    brown_dough.make_pretty()  # shape the dough

brown_dough.prove(hours=2)  # prove the dough

brown_dough.bake(hours=1)

brown_dough.set_price("£1.50")
```